Testing code given at -> https://docs.llamaindex.ai/en/stable/examples/llm/ollama/

In [1]:
from llama_index.llms.ollama import Ollama

In [11]:
model = "phi4:latest"
context_window = 1000

In [2]:
llm = Ollama(
    model=model,
    request_timeout=120.0,
    # Manually set the context window to limit memory usage
    context_window=context_window,
)

In [3]:
%%time
resp = llm.complete("Who is Paul Graham?")

In [4]:
print(resp)

Paul Graham is a well-known computer programmer, writer, and entrepreneur. He co-founded Viaweb with Jessica Livingston in 1996, which was later sold to Yahoo! for $49 million; this company eventually evolved into what we know today as the Ruby on Rails framework.

In addition to his work in software development, Paul Graham is also recognized for his insightful essays on technology and entrepreneurship. He founded Y Combinator, a prestigious startup accelerator, in 2005 alongside Jessica Livingston and Trevor Blackwell. Y Combinator has been instrumental in funding and supporting numerous successful startups, such as Airbnb, Dropbox, and Reddit.

Graham's writings often explore the intersection of programming, creativity, and business. His essays are collected in books like "Hackers & Painters" and "How to Get Rich (Revised and Updated)." Through his work, he has significantly influenced both the tech industry and modern entrepreneurship culture.


In [5]:
from llama_index.core.llms import ChatMessage

messages = [
    ChatMessage(
        role="system", content="You are a pirate with a colorful personality"
    ),
    ChatMessage(role="user", content="What is your name"),
]
resp = llm.chat(messages)

In [6]:
print(resp)

assistant: Ahoy there! I be Captain Redbeard, the most fearsome and merry pirate to sail these briny blues! With me trusty parrot Squawkington by me shoulder and a treasure map always in me grasp, we set off on adventures as grand as the seven seas themselves. And what adventure might bring ye to cross paths with ol' Redbeard today?


In [9]:
%%time
response = llm.stream_complete("Who is Paul Graham?")
for r in response:
    print(r.delta, end="")

Paul Graham is a well-known computer programmer, writer, and entrepreneur. Born in 1973 in New Zealand, he gained significant recognition as one of the founders of Viaweb (later known as Yahoo! Store) with Robert Morris. In 1999, Yahoo acquired Viaweb for approximately $49 million in stock, which marked a notable success in the tech industry.

Paul Graham is also renowned for his influential essays on programming and technology, many of which are collected in "Hackers & Painters," published in 2004. His writing has had a substantial impact on how people think about software development and entrepreneurship.

In addition to his writing, Paul Graham founded Y Combinator in 2005 along with Trevor Blackwell and Jessica Livingston. Y Combinator is an early-stage startup accelerator that has become one of the most successful incubators globally, supporting numerous high-profile companies like Dropbox, Airbnb, Reddit, and Stripe.

Beyond these accomplishments, Paul Graham also established a p

In [12]:

llm = Ollama(
    model=model,
    request_timeout=120.0,
    json_mode=True,
    # Manually set the context window to limit memory usage
    context_window=context_window,
)

In [13]:
%%time
response = llm.complete(
    "Who is Paul Graham? Output as a structured JSON object."
)
print(str(response))

{
  "name": "Paul Graham",
  "aliases": [
    {
      "name": "Y Combinator",
      "role": "Co-founder and former President"
    }
  ],
  "profession": [
    {
      "title": "Entrepreneur",
      "description": "Known for founding Y Combinator, a startup accelerator."
    },
    {
      "title": "Writer",
      "description": "Authored several influential essays on technology and startups."
    },
    {
      "title": "Software Engineer",
      "description": "Involved in various software development projects early in his career."
    }
  ],
  "notable_works": [
    {
      "name": "Y Combinator",
      "description": "A startup accelerator that has helped fund and develop numerous successful companies."
    },
    {
      "name": "Essay Collections",
      "examples": ["Hackers & Painters", "Great Hackers"],
      "description": "Compilations of essays on programming, technology, and innovation."
    }
  ],
  "birth_date": "January 13, 1973",
  "nationality": "American",
  "contribu

### Structured Outputs

In [14]:
from llama_index.core.bridge.pydantic import BaseModel

class Song(BaseModel):
    """A song with name and artist."""
    name: str
    artist: str

In [15]:
sllm = llm.as_structured_llm(Song)

In [18]:
[i for i in dir(sllm) if 'schema' in i]

['__get_pydantic_core_schema__',
 '__get_pydantic_json_schema__',
 '__pydantic_core_schema__',
 'model_json_schema',
 'schema',
 'schema_json']

In [23]:
x = sllm.model_json_schema()

In [24]:
x.keys()

dict_keys(['$defs', 'description', 'properties', 'required', 'title', 'type'])

In [26]:
x['description']

'A structured LLM takes in an inner LLM along with a designated output class,\nand all methods will return outputs in that structure.'

In [28]:
x['required']

['llm', 'output_cls']

In [31]:
%%time
from llama_index.core.llms import ChatMessage

response = sllm.chat([ChatMessage(role="user", content="Name a random song!")])
print(response.message.content)

{"name":"Shape of You","artist":"Ed Sheeran"}
CPU times: user 3.09 ms, sys: 164 μs, total: 3.26 ms
Wall time: 6.36 s


In [35]:
%autoawait

response = await sllm.achat(
    [ChatMessage(role="user", content="Name a random song!")]
)
print(response.message.content)

IPython autoawait is `on`, and set to use `asyncio`
{"name":"Bohemian Rhapsody","artist":"Queen"}


/tmp/ipykernel_20488/3510952307.py:3: RuntimeWarning: coroutine 'Dispatcher.span.<locals>.async_wrapper' was never awaited
  response = await sllm.achat(


In [36]:

response_gen = sllm.stream_chat(
    [ChatMessage(role="user", content="Name a random song!")]
)
for r in response_gen:
    print(r.message.content)

{"name":null,"artist":null}
{"name":null,"artist":null}
{"name":null,"artist":null}
{"name":"","artist":null}
{"name":"Bo","artist":null}
{"name":"Bohem","artist":null}
{"name":"Bohemian","artist":null}
{"name":"Bohemian R","artist":null}
{"name":"Bohemian Rhaps","artist":null}
{"name":"Bohemian Rhapsody","artist":null}
{"name":"Bohemian Rhapsody","artist":null}
{"name":"Bohemian Rhapsody","artist":null}
{"name":"Bohemian Rhapsody","artist":null}
{"name":"Bohemian Rhapsody","artist":null}
{"name":"Bohemian Rhapsody","artist":""}
{"name":"Bohemian Rhapsody","artist":"Queen"}
{"name":"Bohemian Rhapsody","artist":"Queen"}
{"name":"Bohemian Rhapsody","artist":"Queen"}
